# DSP Basics


*This material is a joint work of TAs and the instructor from IC Lab at KAIST, including Woohyeok Choi, Sangjun Park, Yunjo Han, Soowon Kang, Auk Kim, Inyeob Kim, Minhyung Kim, Hansoo Lee, Uichin Lee, Cheul Y. Park, and Eunji Park. This work is licensed under CC BY-SA 4.0.*

## Preparation

In [1]:
!pip install plotly numpy pandas

## Signal: Sinusoidal Waves

Let's generate three cosine waves with different frequencies (i.e., 1, 2, 4 Hz).

As data is created in an array form, it is necessary to set the parameters in relation to data sampling:
* Sampling Rate
* Sampling Duration


In our example, the sampling rate is set as 1024 Hz.

In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

sample_duration = 1 # in seconds
f_s = 4000 # sampling frequency
num_samples = sample_duration * f_s

# An array of sampling time
t = np.arange(num_samples) / f_s

# Make signals with different frequencies
signals = {}
for freq in [1, 2, 4]:
    signals[freq] = np.cos(2 * np.pi * freq * t)


# Visualize each signals
fig = make_subplots(rows=3, cols=1)
for idx, freq in enumerate(signals.keys()):
    fig.add_trace(go.Scatter(x= t, y= signals[freq],
                        mode='lines',
                        name='{} HZ'.format(freq)), row=idx+1, col=1)
fig.update_layout(
    xaxis3 = dict(title = "Time (s)"),
    yaxis2 = dict(title = "Magnitude")
)

fig.show()

## Sampling Sinusoidal Waves

Let's sample our high resolution sinusodial waves.

In reality, if a continuous signal is measured through a sensor, it will be recorded as discrete signal depending on the sampling rate.

Assumed that, the signal we created above is continuous singal, and sample above signals at 5Hz.

In [3]:
# Because original signal is sampling with 2000HZ, we will pick sample every 400s.

fig = make_subplots(rows=3, cols=1,  shared_xaxes = True)
for idx, freq in enumerate([1,2,4]):
    fig.add_trace(go.Scatter(x=t, y=signals[freq],
                    mode='lines',
                    name='{} HZ'.format(freq)), row=idx+1, col=1)
    # Mark Resampling
    fig.add_trace(go.Scatter(x=t[::f_s//5], y= signals[freq][::f_s//5],
                    mode='markers', marker = {'size':10, 'color':'black'}, name='sampling', showlegend = (idx == 2)), row=idx+1, col=1)


fig.update_layout(
    xaxis3 = dict(title = "Time (s)"),
    yaxis2 = dict(title = "Magnitude")
)

fig.show()

## Nyquist Sampling Rate

In the class, we learned that the sampling rate should be greater than two times the bandwidth.

Let's see what happens if we sample the 4Hz sinusoidal signal at 1, 2, 4, 8, and 16 Hz sampling rate


We can see that 1, 2, and 4 Hz seems like constant wave which means that these are not sufficient sampling rate to capture 4 Hz sinusoidal wave.

In [4]:
signal = signals[4]


fig = make_subplots(rows=5, cols=1,  shared_xaxes = True)
# For each sampling rates
for idx, freq in enumerate([1,2,4,8,16]):
    fig.add_trace(go.Scatter(x= t, y = signal,
                            mode = 'lines', marker = {'color': '#FF9999'}, name = "4 Hz Original", showlegend = (idx == 0)), row = idx+1, col = 1)
    fig.add_trace(go.Scatter(x= t[::f_s//freq], y = signal[::f_s//freq],
                            mode= "lines+markers", marker = {'size':10, 'color': '#666'}, name= "Sampling at {} Hz".format(freq)), row = idx+1, col = 1)

fig.update_layout(
    xaxis5 = dict(title = "Time (s)"),
    yaxis3 = dict(title = "Magnitude")
)
fig.show()

## Aliasing
Let's visualize a 20 Hz cosine wave, and then sample the signal at different rates: 8, 16, 32, 40 Hz.

Interestingly, when sampled at low rates (i.e., 8, 16, and 32 Hz), the resulting sampled signals looked very much different from the orignal signal (i.e., 20 Hz cosine wave); these signals appear to be 4 Hz -- indeed, these signals are aliases of the orignal signal.

In [5]:
signal = np.cos(2 * np.pi * 20 * t)

fig = make_subplots(rows=4, cols=1,  shared_xaxes = True)
# Plot according to different sampling rates
for idx, freq in enumerate([8, 16, 32, 40]):
    fig.add_trace(go.Scatter(x= t, y = signal,
                            mode = 'lines', marker = {'color': '#FF9999'}, name = "20 Hz Original", showlegend = (idx == 0)), row = idx+1, col = 1)
    fig.add_trace(go.Scatter(x= t[::f_s//freq], y = signal[::f_s//freq],
                            mode= "lines+markers", marker = {'size':10, 'color': '#666'}, name= "Sampling at {} Hz".format(freq)), row = idx+1, col = 1)

fig.update_layout(
    xaxis4 = dict(title = "Time (s)"),
    yaxis3 = dict(title = "Magnitude")
)
fig.show()

## Mini-exercise #1
So far we have tested a cosine wave. Please generate a high frequency sine wave: 100 Hz. Test how different sampling frequencies affect the results: 20 Hz, 50 Hz, 100 Hz, 150 Hz.

<img src="https://drive.google.com/uc?id=1TPp-uTV0DwzymBIGXtpr4vdBaALp3AxM"/>

In [6]:
# Code given
freq = 6000 # original signal was sampled in 6000Hz
times = np.arange(0, .1, 1/freq)
signal = np.sin(2 * np.pi * 100 * times)

# Write your answer below

## DFT: Discrete Fourier Transform
In this exercise, we'll visually examine how DFT works. This content is based on [this blog article](http://practicalcryptography.com/miscellaneous/machine-learning/intuitive-guide-discrete-fourier-transform/).




### Step 1: Visualize a original signal
Let's first generate a new original signal which has 4HZ cosine signal and 10HZ consine signal wih random noise.

$$\cos{2\pi f_1 t} + 0.3\cos{2\pi f_2t} + \epsilon, \epsilon \sim N(0, .2^2)$$

$$f_1 = 4, f_2 = 10 $$

In [7]:
import plotly.express as px

f_s = 2000
duration = 1
t = np.arange(0,duration, 1/f_s)

# Set seed to fix the normal signal
np.random.seed(2023)
signal = np.cos(2*np.pi* 4 * t) + .3 * np.cos(2 * np.pi * 10 * t) + np.random.normal(0, .2, t.shape)

fig = px.line(x= t, y= signal)
fig.update_layout(
    xaxis = dict(title = "Time (s)"),
    yaxis = dict(title = "Magnitude", range = [-3,3])
)
fig.show()


### Step 2. Generate a sample wave

Let's assume we sample the original signal at 100 Hz.
It will look like the graph that will be shown in the result below.

In [8]:
sampling_freq = 100
sampling_signal = signal[::f_s//sampling_freq]
sampling_t = t[::f_s//sampling_freq]

fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True)

fig.append_trace(go.Scatter(x = t, y = signal, mode= "lines", name="Original"),row = 1, col= 1)
fig.append_trace(go.Scatter(x = t[::f_s//sampling_freq], y = signal[::f_s//sampling_freq], mode= "lines+markers", name="Sampling"),row = 2, col= 1)

fig.update_layout(
    xaxis2 = dict(title = "Time (s)")
)
fig.show()

### Step 3: Calculate Correlation for DFT

To determine how much each frequency component appears in a signal, we will calculate the correlation between the signal and a sinusoidal wave with that frequency.

Correlation of two signals can be calculated as below:
$$\sum_{n=0}^{N-1}x(n)y(n)$$

Freqeuncy with $\frac{k}{N}f_s (k = 0,1, \dots N)$ will be calculated by correlation with sinusoidal signal:
$$\cos{\frac{2\pi k n}{N}}$$

Although it may be necessary to derive the Discrete Fourier Transform from the Continuous Fourier Transform for a precise explanation of $\frac{k}{N}$, let's explain it simply. If the period of the Sampling Signal is $f_s$ and to fully capture one period through N samples, the frequency should be $\frac{k}{N}f_s$.

In [9]:
N = len(sampling_signal)
fig = make_subplots(rows = 5, cols =1)

# Show graphs according to each frequency
for k in range(5):
    cos_signal = np.cos(2*np.pi*k*np.arange(0, 1,1/N))
    corr = np.sum(cos_signal * sampling_signal)
    fig.append_trace(
        go.Scatter(x = sampling_t, y = sampling_signal, mode = "lines+markers", name = "sampling signal", marker = dict(color = "black"), showlegend = (k == 0)), row = k+1, col = 1
    )
    fig.append_trace(
        go.Scatter(x = sampling_t, y = cos_signal, mode = "lines+markers", name = "{} HZ".format(k/N * sampling_freq)), row = k+1, col = 1
    )
    fig.update_layout(**{
        "xaxis"+ ("" if k==0 else str(k+1)): dict(title = "corr: {}".format(corr))
    })

fig.update_layout(
    height = 1200
)
fig.show()

We use both **`cos()`** and **`sin()`** to calculate correlation to get phase of that frequency component. Thus, frequency including phase can be represented as a complex number as shown below:

$$X(k) = \sum_{n=0}^{N-1}x(n) \cos{\frac{2\pi k n}{N}} - i\sum_{n=0}^{N-1}x(n)\sin{\frac{2\pi k n}{N}}$$

And then, we will visualize the magnitude and phase of $X(k)$.


The Nyquist sampling rate states that the sampling rate must be greater than twice the signal's frequency, which means that only frequency components less than half the sampling rate (50 Hz in this case) will be valid.

From the graph, we can see that the sampled signal is composed of sinusoidal waves with frequencies of 4 Hz and 10 Hz.

In [10]:
X = []
for k in range(N):
    cos_signal = np.cos(2*np.pi*k*np.arange(0, 1, 1/N))
    sin_signal = np.sin(2*np.pi*k*np.arange(0, 1, 1/N))
    # we use j to denote an imaginary number
    X.append(np.sum(cos_signal * sampling_signal) - 1j * np.sum(sin_signal * sampling_signal))

magnitude = np.absolute(X)
freq = np.arange(N)/N*sampling_freq

# Remaining phase only for peak
phase = np.angle(X)
phase[magnitude < 5] = 0
# Radian to Degree
phase *= 180/np.pi

'''
Typically, k larger than N/2 is illustrated as -k because
hey are meaningless (nyquist sampling rate) and
magnitude is even and phase is odd function with respect to N/2.
'''
freq = [*(freq[N//2:]-sampling_freq), *(freq[:N//2])]
magnitude = [*magnitude[N//2:], *(magnitude[:N//2])]
phase = [*phase[N//2:], *(phase[:N//2])]


fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True)

fig.append_trace(
    go.Scatter(x = freq, y = magnitude, mode = "lines+markers", name="magnitude")
    , row =1, col = 1
)

fig.append_trace(
    go.Scatter(x = freq, y = phase, mode = "lines+markers", name="phase")
    , row =2, col = 1
)
fig.update_layout(
    xaxis2 = dict(title = "Frequency (HZ)")
)
fig.show()

## Mini-exercise #2
Let's compare the result of the figure for an example we manually calculated DFT values in the class:

* Let's assume a basic 1 Hz signal: $\cos(2\pi t)$
* We sample this signal at a frequency of $f_s = 4 \text{ Hz}$, resulting in 4 samples that start at $t = 0$.
* Then, we increase the number of samples, meaning that we observe a longer period of time (t=1, 2, 3 seconds), whch corresponds to 4, 8, 12 samples.  

Before we do that, please use the following DFT function. Unlike our previous example, we calculate DFT using the original equation.

Please answer the questions below:

In [11]:
# Code given
# sampling_freq - sampling frequency
# signal = sampled signals in an array
def DFT(sampling_freq, signal):
    n = len(signal)

    # Q1 - what is freq_delta?
    freq_delta = sampling_freq / n
    freq = np.arange(n) * freq_delta

    # Calculate DFT
    spectrum = np.zeros(n, dtype=complex)
    for k in range(n):
        for j in range(n):
            spectrum[k] += signal[j] * np.exp(-2j * np.pi * k * j / n)

    magnitude = np.absolute(spectrum)
    phase = np.angle(spectrum) * 180/np.pi

    # Q2 - the following implements fftshift: what is it? and why are we doing this?
    freq = [*(freq[n//2:]-sampling_freq), *freq[:n//2]] # asterisk * and ** are unpacking operators in Python
    magnitude = [*magnitude[n//2:], *magnitude[:n//2]]
    phase = [*phase[n//2:], *phase[:n//2]]

    return freq, magnitude, phase

Now, let's plot the graphs and answer the following questions:
* Q3 - As you increase the number of samples, what patterns do you observe?
* Q4 - Try to double the sampling frequency: i.e., f_s = 8. You need to also vary the number of samples to meet with changed sampling frequency. What patterns do you observe?

In [12]:
f_s = 4
# Create a 2-row, 3-column subplot layout with subplot titles (titles are added for the first row subplots)
fig = make_subplots(
    rows=2,
    cols=3,
    shared_xaxes=True,
    subplot_titles=['n_sample: 4', 'n_sample: 8', 'n_sample: 12']
)

for idx, n_sample in enumerate([4, 8, 12]):
    freq, magnitude, phase = DFT(sampling_freq=f_s, signal=np.cos(2 * np.pi * np.arange(n_sample) / f_s))

    # Add magnitude plot to row 1
    fig.add_trace(
        go.Scatter(x=freq, y=magnitude, mode="markers", name=f"Mag (N={n_sample})", showlegend=False),
        row=1, col=idx+1
    )

    # Add phase plot to row 2
    fig.add_trace(
        go.Scatter(x=freq, y=phase, mode="markers", name=f"Phase (N={n_sample})", showlegend=False),
        row=2, col=idx+1
    )

fig.update_layout(
    title="DFT Magnitude and Phase Plots",
    # Update y-axis title for the first subplot in row 1 (magnitude) and row 2 (phase)
    yaxis=dict(title="Magnitude"),
    yaxis4=dict(title="Phase")  # yaxis4 corresponds to row 2, col 1
)
fig.show()

# If you want to connect the points with a line, you can change the mode to "markers+lines"
# go.Scatter(x=freq, y=magnitude, mode="markers+lines", name="Magnitude")


## FFT - Fast Fourier Transform (DFT)  

We can manually calculate DFT, but it can be calculated faster using the function called FFT.
This function is provided by numpy.

<!-- Instead of doing DFT calculations as we did earlier, we'll simply use the fft function defined in Numpy; there are also inverse fft functions. You need to learn how to interpret fft results. Please quickly go over the manuals of the following three functions. Why do we need fftshift and fftfreq? You should be able to answer these questions after learning the basic materials in the class.

* [fft](https://docs.scipy.org/doc/numpy/reference/routines.fft.html)
* [fftshift](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.fft.fftshift.html)   
* [fftfreq](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftfreq.html )   -->

In [13]:
f_s = 2000
duration = 1
t = np.arange(0,duration, 1/f_s)

# Set seed to fix the normal signal
np.random.seed(2023)
signal = np.cos(2*np.pi* 4 * t) + .3 * np.cos(2 * np.pi * 10 * t) + np.random.normal(0, .2, t.shape)

sampling_rate = 100
sampling_signal = signal[::f_s//sampling_rate]

Xk = np.fft.fft(sampling_signal)
freq = np.fft.fftfreq(len(sampling_signal),1/sampling_rate)

'''
Typically, k larger than N/2 is illustrated as -k because
hey are meaningless (nyquist sampling rate) and
magnitude is even and phase is odd function with respect to N/2.
'''
Xk_shifted = np.fft.fftshift(Xk) # Shift zero freq to center
freq_shifted = np.fft.fftshift(freq) # Shift zero freq to center

fig = make_subplots(rows=4, cols=1, vertical_spacing=0.05)

fig.add_trace(go.Scatter(x=t, y=signal, mode='lines', name="original"), row=1, col=1)

fig.add_trace(go.Scatter(x=t[::f_s//sampling_rate], y=sampling_signal, mode='lines+markers', name="sampling signal"), row=2, col=1)

fig.add_trace(go.Scatter(x=freq_shifted, y=np.absolute(Xk_shifted),
                    mode='lines+markers',
                    name='Magnitude'), row=3, col=1)

phase = np.angle(Xk_shifted)
phase[np.absolute(Xk_shifted) < 5] = 0

fig.add_trace(go.Scatter(x=freq_shifted, y=phase*180/np.pi,
                    mode='lines+markers',
                    name='Phase'), row=4, col=1)

fig.update_layout(
    xaxis = dict(range = [0,1]),
    xaxis2 = dict(range = [0,1]),
    xaxis3 = dict(range = [-51,51]),
    xaxis4 = dict(range = [-51,51]),
)

fig.show()

## Zero Padding / Spectral Resolution
Zero padding helps increase spectral resolution. We'll use **`np.pad()`** function in Numpy.

Please carefully compare the plots; what happens when we increase the zero padding size?






In [14]:
# zero padding

# pad in front of signal
fig = make_subplots(rows=2, cols=3, start_cell="top-left", shared_xaxes = True)
for idx, val in enumerate([0,50,100]):
    sampling_signal_pad = np.pad(sampling_signal, (val,0))
    X_k = np.fft.fftshift(np.fft.fft(sampling_signal_pad))
    freq = np.fft.fftshift(np.fft.fftfreq(len(sampling_signal_pad), 1/sampling_rate))

    fig.add_trace(go.Scatter(x=freq, y=np.absolute(X_k),
                        mode='lines+markers',
                        name='Magnitude',showlegend = False), row=1, col=idx+1)
    phase = np.angle(X_k)
    # phase[np.absolute(X_k) < 15] = 0
    fig.add_trace(go.Scatter(x=freq, y=phase*180/np.pi,
                        mode='lines+markers',showlegend = False,
                        name='Phase'), row=2, col=idx+1)
fig.update_layout(
    xaxis4  = dict(title="Original"),
    xaxis5  = dict(title="Pad with 100 zeros"),
    xaxis6  = dict(title="Pad with 500 zeros"),

    yaxis = dict(title = "Magnitude"),
    yaxis4 = dict(title = "Phase")
)

fig.show()

# Homework #3

*   3 points towrad your final score.
*   Homework #3 is due on **3/26 at 23:59:59**.

## Q1 Comparing DFT results of different sampling rate (2 points)
Compare the result of different sampling rate.

* Resample the signal **`x`** given below with 40HZ and 160HZ. (0.4 points, 0.2 point for each signal)
* Generate the Freuqency Spectrum plot including magnitude and phase for each resampled signal. (0.6 point, 0.15 point for each plot)
    * *Please remove the phase of frequency which is not the peak*

* Please describe the difference between two resampled signals, and explain why that difference occurred. (1.0 point)
    * Please provide detailed information on why the frequency peak occurs at 10Hz when sampled at 40Hz.
    * In the context of sampling with 40Hz, please explain the ratio of magnitude between peaks at 5Hz and 10Hz, as well as the phase of each peak.


<img src="https://drive.google.com/uc?id=1pzGO4OtnkbHOtYD-JcW1uJCW06YRrCVZ"/>

In [15]:
# Given Code
original_freq = 8000

np.random.seed(2023)
t = np.arange(original_freq)/original_freq
x = np.cos(2*np.pi * 5* t) + 0.2 * np.cos(2*np.pi *30 * t + np.pi) + np.random.normal(0, .1, t.shape)

# Write your code below

## Q2. Implementing IDFT(1 point)
Please implement inverse DFT(Do not use libraries that provide `np.fft.ifft()`!) and plot original graph from magnitude and phase of freqeuncy spectrum.

* Implement IDFT function that input is unshifted magnitude and phase, and return inverted signal. (0.4 point)
* Please apply the inverse DFT into two resampled signal in Q1, and draw the plot below. (0.4 point)
* Compare the IDFT you implemented to `np.fft.ifft()` using two sampled signal in Q1 by calculating Mean Squared Error(MSE) of resulted signal. (0.2 point)
    * You should ensure that MSE result is very close to zero using `np.isclose()`.

<img src="https://drive.google.com/uc?id=1IzMtkPDYGDGNwWB4uZU_OfSlAiWyScTz"/>


In [16]:
# Implement your inverse DFT function

In [17]:
# Plot the graphs and compare them